<a href="https://colab.research.google.com/github/mok3bat/DataDevQuest/blob/main/DataDevQuest_2025_01_Advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# Install Tableau Server Client (TSC)
!pip install tableauserverclient PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.3 MB/s eta 0:00:00


# **Login to Tableau Cloud Site**

In [6]:
import tableauserverclient as TSC
from google.colab import userdata


my_server_url = 'https://10ax.online.tableau.com/'
my_token_name = userdata.get('TC_token_name')
my_token_secret = userdata.get('TC_token_secret')
my_site_id = userdata.get('TC_site_id')
my_workbook = "Let's Play A Game"


tableau_auth = TSC.PersonalAccessTokenAuth( token_name=my_token_name
                                           ,personal_access_token=my_token_secret
                                           ,site_id=my_site_id)
server = TSC.Server(my_server_url, use_server_version=True)

# Sign in to Tableau Cloud

server.auth.sign_in_with_personal_access_token(tableau_auth)
print('[Logged in successfully to {}]'.format(my_server_url))

[Logged in successfully to https://10ax.online.tableau.com/]


# **Retrieve only the requested workbook and Export as PDF**

In [30]:
import os
from google.colab import files
import PyPDF2
from PyPDF2 import PdfMerger
import io
import pandas as pd


# Add filteration clause to consider only the requested workbook
req_option = TSC.RequestOptions()
req_option.filter.add(TSC.Filter(TSC.RequestOptions.Field.Name
                                ,TSC.RequestOptions.Operator.Equals
                                ,my_workbook))


# Print workbook details
workbooks, pagination_item = server.workbooks.get(req_options=req_option)
if workbooks:
  wb = workbooks[0]
  print(f"=== Workbook ===")
  print(f"Name:{wb.name}")
  print(f"luid:{wb.id}")
  print(f"URL:{wb.webpage_url}")
  print(f"View:{wb.content_url}")
  print("=================")
  print("\n")
  my_content_url = wb.content_url
  pdf_file_name = f'/content/sample_data/{my_content_url}.pdf'
  pdf_options = TSC.PDFRequestOptions(page_type='unspecified', orientation='landscape', maxage=1)

  #Read Publishers' names
  df=pd.read_csv('/content/sample_data/SelectedPublisher.csv')

  # List of PDF files you want to merge
  pdf_files = []

  # Generate the filtered views
  for series_name, series in df.items():
    for pub in series.values:
      # set a view filter
      print(f"===Processing Publisher:\t{pub}")
      pdf_options.vf('SelectedPublisher', pub)
      server.workbooks.populate_pdf(wb, pdf_options)
      # Convert populated bianry pdf file to be readable by the merger.
      wb_temp = io.BytesIO(wb.pdf)
      pdf_files.append(wb_temp)

  # Create a merger object
  merger = PdfMerger()

  # Merge generated PDFs
  for pdf in pdf_files:
      merger.append(pdf)

  # Export the Consolidated PDF
  merger.write(f"/content/sample_data/{my_content_url}.pdf")
  merger.close()
  print('[PDF Generated]')


=== Workbook ===
Name:Let's Play A Game
luid:f79704b8-e770-4398-bc42-8545c54b2cb4
URL:https://10ax.online.tableau.com/#/site/setitsandboxdev427435/workbooks/1073039
View:LetsPlayAGame


===Processing Publisher:	Activision
===Processing Publisher:	Electronic Arts
===Processing Publisher:	Konami Digital Entertainment
===Processing Publisher:	Namco Bandai Games
===Processing Publisher:	Nintendo
===Processing Publisher:	Sega
===Processing Publisher:	Sony Computer Entertainment
===Processing Publisher:	Take-Two Interactive
===Processing Publisher:	THQ
===Processing Publisher:	Ubisoft
[PDF Generated]


# **Sign out from Tableau Cloud**

In [ ]:
# Sign Out from Tableau Cloud

server.auth.sign_out()

# **Refrences:**

1. https://help.salesforce.com/s/articleView?id=001458516&type=1
2. https://tableau.github.io/server-client-python/docs/api-ref#pdfrequestoptions-class

